In [54]:
from datasets import load_dataset #type: ignore
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [ ]:
#code for the model, the architecture of the model is defined here
#input features: q_type, wpm, idle, latency, mouse_movement
#hidden layer 1: 5 nodes
#hidden layer 2: 5 nodes
#hidden layer 3: 2 nodes
#output logits layer: 3 nodes, where classes are low risk, mild risk, high risk

In [116]:
g = torch.Generator().manual_seed(2147)   #setting the seed for reproducibility  

In [162]:
# Load the dataset
df = pd.read_csv('typing_risk_data_1000.csv')

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert categorical labels to numerical values
risk_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
y = df['Risk Level'].map(risk_mapping).values  # Shape: (batch_size,)

# Extract the first two columns as features
X = df.iloc[:, :2].values  # Shape: (batch_size, 2)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

# Compute mean and standard deviation for each feature
mean = X.mean(dim=0)
std = X.std(dim=0)

# Apply Z-score normalization
X_transformed = (X - mean) / std

print("Transformed Feature shape:", X_transformed.shape)
print("Target shape:", y.shape)

Transformed Feature shape: torch.Size([999, 2])
Target shape: torch.Size([999])


In [163]:
X.shape

torch.Size([999, 2])

In [164]:
y.shape

torch.Size([999])

In [165]:
#random initialization of weights and biases
W1 = torch.randn((2, 5), generator=g, requires_grad=True)
b1 = torch.randn((5, ), generator=g, requires_grad=True)

W2 = torch.randn((5, 5), generator=g, requires_grad=True)
b2 = torch.randn((5, ), generator=g, requires_grad=True)

W3 = torch.randn((5, 2), generator=g, requires_grad=True)
b3 = torch.randn((2, ), generator=g, requires_grad=True)

W4 = torch.randn((2, 3), generator=g, requires_grad=True)
b4 = torch.randn((3, ), generator=g, requires_grad=True)


In [166]:
parameters = [W1, b1, W2, b2, W3, b3, W4, b4]

In [167]:
#total num of params for bookkeeping
sum(p.nelement() for p in parameters) 

66

In [168]:
def train(iters, alpha=0.01):
    """
    function to train the model, uses stochastic gradient descent
    inputs: iters - number of iterations to train the model
    """

    for step in range(iters):
        #forward pass
        l1 = X@W1 + b1  #shape: (batch_size, 10)
        a1 = F.relu(l1) #shape: (batch_size, 10)

        l2 = a1@W2 + b2  #shape: (batch_size, 10)
        a2 = F.relu(l2)

        l3 = a2@W3 + b3  #shape: (batch_size, 5)
        a3 = F.relu(l3)

        logits = a3@W4 + b4  #shape: (batch_size, 3)
        neg_log_loss = F.cross_entropy(logits, y)

        print(f"loss at iter {step+1}: {neg_log_loss.item()}")

        #backward pass
        for p in parameters:
            p.grad = None  #resetting the gradients, to avoid gradient accumulation
        neg_log_loss.backward()

        #update params
        for p in parameters:
            p.data -= alpha*p.grad  #SGD update        

In [169]:
train(200)

loss at iter 1: 24.732757568359375
loss at iter 2: 1.1917325258255005
loss at iter 3: 1.1912716627120972
loss at iter 4: 1.190812587738037
loss at iter 5: 1.1903555393218994
loss at iter 6: 1.189900279045105
loss at iter 7: 1.1894465684890747
loss at iter 8: 1.1889948844909668
loss at iter 9: 1.1885451078414917
loss at iter 10: 1.1880970001220703
loss at iter 11: 1.1876507997512817
loss at iter 12: 1.187206506729126
loss at iter 13: 1.1867635250091553
loss at iter 14: 1.1863229274749756
loss at iter 15: 1.18588387966156
loss at iter 16: 1.1854466199874878
loss at iter 17: 1.1850112676620483
loss at iter 18: 1.1845778226852417
loss at iter 19: 1.1841456890106201
loss at iter 20: 1.1837158203125
loss at iter 21: 1.183287262916565
loss at iter 22: 1.1828609704971313
loss at iter 23: 1.1824359893798828
loss at iter 24: 1.1820130348205566
loss at iter 25: 1.1815918684005737
loss at iter 26: 1.1811723709106445
loss at iter 27: 1.180754542350769
loss at iter 28: 1.1803385019302368
loss at ite

In [170]:
@torch.no_grad()
def out_logits(X):
        l1 = X@W1 + b1
        a1 = F.relu(l1)

        l2 = a1@W2 + b2  
        a2 = F.relu(l2)

        l3 = a2@W3 + b3 
        a3 = F.relu(l3)

        logits = a3@W4 + b4  
        return logits

In [171]:
logits_1 = out_logits(X)
probs = F.softmax(logits_1, dim=1)  #final probabilities of the classes

max_probs, max_indices = torch.max(probs, dim=1)  #get the max prob class
print(max_indices)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [182]:
p = probs.tolist()

In [183]:
type(p)

list

In [184]:
p

[[0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923, 0.22292742133140564],
 [0.3859386444091797, 0.3911339044570923

In [111]:
print(y)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [185]:
torch.save({'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3, 'W4': W4, 'b4': b4}, "model_params.pth")